# Planet API Python Client


This tutorial is an introduction to [Planet](https://www.planet.com)'s Data and Orders API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `planet` module. It shows you how to create bulk orders, use our tools, and deliver to the cloud.

## Requirements

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Python modules used in this tutorial are:
* [IPython](https://ipython.org/) and [Jupyter](https://jupyter.org/)
* [planet](https://github.com/planetlabs/planet-client-python)
* [geojsonio](https://pypi.python.org/pypi/geojsonio)
* [rasterio](https://rasterio.readthedocs.io/en/latest/index.html)
* [asyncio](https://docs.python.org/3/library/asyncio.html)

You should also have an account on the Planet Platform and retrieve your API key from your [account page](https://www.planet.com/account/).

## Useful links 
* [Planet Client V2 Documentation](https://github.com/planetlabs/planet-client-python)
* [Planet Data API reference](https://developers.planet.com/docs/apis/data/)

This tutorial will cover the basic operations possible with the Python client, particularly those that interact with the Data API and Orders API

## Set up

In order to interact with the Planet API using the client, we need to import the necessary packages & define helper functions.

In [ ]:
#general packages
import os
import json
import glob
import asyncio
import requests
import numpy as np
import nest_asyncio 
import contextily as ctx
import matplotlib.pyplot as plt
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta

#geospatial packages
import rasterio
from rasterio import features
import geopandas as gpd


#planet SDK
from planet import Auth, reporting, Session, OrdersClient, order_request, data_filter


# We will also create a small helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

We next need to create a `client` object registered with our API key. The API key will be automatically read from the `PL_API_KEY` environment variable if it exists. If not, you can provide it below. You can also authenticate via the CLI using [`auth init`](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/cli/cli-reference/?h=auth#auth:~:text=message%20and%20exit.-,auth,-%C2%B6), this will store your API key as an environment variable.

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if 'PL_API_KEY' in os.environ:
    API_KEY = os.environ['PL_API_KEY']
else:
    API_KEY = 'PASTE_API_KEY_HERE'
    os.environ['PL_API_KEY'] = API_KEY

client = Auth.from_key(API_KEY)

## Searching

We can search for items that are interesting by using the `quick_search` member function. Searches, however, always require a proper request that includes a filter that selects the specific items to return as seach results.

Let's also read in a GeoJSON geometry into a variable so we can use it during testing. The geometry can only have one polygon to work with the data API

In [ ]:
def plot_aoi(gdf):
    gdf = gdf.to_crs(epsg=3857)  # reproject it in Web mercator
    ax = gdf.boundary.plot()

    # Add an openstreet map
    ctx.add_basemap(ax, url=ctx.providers.OpenStreetMap.Mapnik.url)
    ax.set_axis_off()
    plt.show()

In [ ]:
# GEOMETRY
gdf = gpd.read_file("ucr_fields.geojson")
geom = json.loads(gdf.to_json())["features"][0]["geometry"]

plot_aoi(gdf)

In [ ]:
json.loads(gdf.to_json())

### Filters

The possible filters include `and_filter`, `date_range_filter`, `range_filter` and so on, mirroring the options supported by the Planet API. Additional filters are described [here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/#filter:~:text=(main())-,Filter,-%C2%B6).

In [ ]:
# Define the filters we'll use to find our data

item_types = ["PSScene"]

#Geometry filter
geom_filter = data_filter.geometry_filter(geom)

#Date range filter
date_range_filter = data_filter.date_range_filter(
    "acquired", gt = datetime(month=12, day=31, year=2022),
    lt = datetime(month=12, day=12, year=2023))

#Clear pixel filter
#clear_filter = data_filter.range_filter('clear_percent', gt = 80)

#Combine all of the filters
combined_filter = data_filter.and_filter([geom_filter, date_range_filter])#, clear_cover_filter])

In [ ]:
combined_filter

#### Directly pinging the API

In [ ]:
item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

In [ ]:
# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(API_KEY, ''),
    json=search_request)

# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]
print(len(image_ids))

Pagination links

In [ ]:
search_result.json()['_links']

#### Using the SDK

In [ ]:
async with Session() as sess:
    cl = sess.client('data')
    item_list = [i async for i in cl.search(search_filter=combined_filter, item_types=item_types,limit=2000)]

If the number of items requested is more than 250, the client will automatically fetch more pages of results in order to get the exact number requested.

Then we can save the output to be visualized as a geojson

In [ ]:
len(item_list)

Inspecting the metadata

In [ ]:
item_list[0]

In [ ]:
#Cloud cover filter
cloud_cover_filter = data_filter.range_filter('clear_percent', gt = 90)

#Combine all of the filters
combined_filter = data_filter.and_filter([geom_filter, date_range_filter, cloud_cover_filter])

async with Session() as sess:
    cl = sess.client('data')
    item_list = [i async for i in cl.search(search_filter=combined_filter, item_types=item_types,limit=500)]

In [ ]:
len(item_list)

Now, we can iterate through our search results.

In [ ]:
for item in item_list:
    print(item['id'], item['properties']['item_type'], item['properties']['clear_percent'])

image_ids = [feature['id'] for feature in item_list]   

## Ordering

### Place a Order
Create the order structure using `planet` functions

In [ ]:
async def assemble_order(name,item_ids):
    products = [
        order_request.product(item_ids, 'analytic_udm2', 'PSScene')
    ]
    
    clip = order_request.clip_tool(aoi=geom_all)
    bandmath = order_request.band_math_tool(b1="b1",
                                            b2="b2",
                                            b3="b3",
                                            b4="b4",
                                            b5="(b4-b3)/(b4+b3)*1000+1000",
                                            pixel_type='16U')

    
    
    tools = [clip,bandmath]

    request = order_request.build_request(
        name, products=products, tools=tools)
    return request
    
request =  await assemble_order("test",image_ids[:2])

In [ ]:
print(request)

Lets create a funcion to order imagery

In [ ]:
async def do_order(request):
    async with Session() as sess:
        cl = OrdersClient(sess)
        with reporting.StateBar(state='creating') as bar:
            order = await cl.create_order(request)
            bar.update(state='created', order_id=order['id'])

            await cl.wait(order['id'],max_attempts=0, callback=bar.update_state)
        os.mkdir(request['name'])
        
        # if we get here that means the order completed. Yay! Download the files.
        await cl.download_order(order['id'],directory=request['name'])


Now we can create all our orders

In [ ]:
await do_order(request)

Now lets visualize our output!

In [ ]:
files = []

files.extend(glob.glob("test/*/*/*bandmath.tif"))
files

# Read the file with Rasterio
with rasterio.open(files[0]) as src:
    print(src.meta)
    # From the open src image, let's save the array as a variable.
    arr = src.read()/10000

In [ ]:
# Order of the bands for 8-band planetscope imagery is from lowest to highest wavelength
band_order = ["blue", "green", "red", "nir","NDVI"]

# Define the layout of our figure
nrow = 1
ncol = 5
f, axes = plt.subplots(nrow, ncol, figsize=(5*ncol, 3.5*nrow))

# Loop over every band name and axis available in the plot together
for band_name, ax in zip(band_order, axes.flatten()):
    # Extract the band from our array
    band_index = band_order.index(band_name)
    band = arr[band_index]

    # Plot in greyscale to the subplot
    im = ax.imshow(band, cmap="Greys_r")
    f.colorbar(im, ax=ax)

    # Set the title for the subplot
    ax.set_title(band_name)

In [ ]:
# Save R, G, and B as variables
red = arr[band_order.index("red")]
green = arr[band_order.index("green")]
blue = arr[band_order.index("blue")]
nir = arr[band_order.index("nir")]

# Stack them into a single array
img = np.stack([red, green, blue], axis=2)
nir_img = np.stack([nir, green, blue], axis=2)

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(8, 3))
axes[0].imshow(img)
axes[0].set_title("RGB")
axes[1].imshow(nir_img)
axes[1].set_title("False Color")

### Google Earth Engine Delivery

In order to deliver straight to earth engine you need to connect your gee account to a google cloud project and authorize planet to deliver to it. [Here](https://developers.planet.com/docs/integrations/gee/quickstart/) are some instrucitons to help with the process.

In [ ]:
async def assemble_gee_order(item_ids, name):
    products = [
        order_request.product(item_ids, 'analytic_sr_udm2', 'PSScene')
    ]
    
    tools = [order_request.clip_tool(aoi=geom)]
    
    delivery = order_request.google_earth_engine(
        project="planet-services-staging",
        collection="gis-day-gee-demo")
    
    request = order_request.build_request(
        name=name, products=products, tools=tools, delivery=delivery)
    return request

In [ ]:
item_ids = [item["id"] for item in item_list]
name = 'gee_delivery_UCR'
request =  await assemble_order(item_ids, name)


# an async Orders client to request order creation
async with Session() as sess:
    cl = OrdersClient(sess)
    with reporting.StateBar(state='creating') as bar:
        # create order via Orders client
        order = await cl.create_order(request)
        bar.update(state='created', order_id=order['id'])
        await cl.wait(order['id'], callback=bar.update_state)

### Google Cloud Delivery
You will need your credentials encoded as base64 with the right accees credentails to your bucket as explained [here](https://developers.planet.com/apis/orders/delivery/#:~:text=Preparing%20Your%20Google%20Cloud%20Storage%20Credentials%C2%B6).

In [ ]:
async def assemble_gcp_order(item_ids, name):
    products = [
        order_request.product(item_ids, 'analytic_sr_udm2', 'PSScene')
    ]
    
    tools = [order_request.clip_tool(aoi=geom)]
    
    delivery = order_request.google_cloud_storage(
        bucket= "gcp_bucket", 
        credentials="base64-encoded-credentials", 
        path_prefix = "")
    
    request = order_request.build_request(
        name=name, products=products, tools=tools, delivery=delivery)
    return request